In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class RecommendationSystem:
    def __init__(self, file_path):
        self.file_path = file_path
        self.df_rooms = self.load_rooms()
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = None
        self.update_tfidf()

    def load_rooms(self):
       
        df = pd.read_excel(self.file_path, sheet_name="Sheet1")
        df["Combined_Features"] = df[["Skills", "Position", "Tracks"]].fillna('').agg(' '.join, axis=1)
        return df

    def update_tfidf(self):
        self.tfidf_matrix = self.vectorizer.fit_transform(self.df_rooms["Combined_Features"])

    def add_new_room(self, room_id, name, skills, position, tracks):
        new_room = pd.DataFrame({
            "ID": [room_id], "Name": [name], "Skills": [skills], "Position": [position], "Tracks": [tracks]
        })
        new_room["Combined_Features"] = new_room[["Skills", "Position", "Tracks"]].fillna('').agg(' '.join, axis=1)
        
        self.df_rooms = pd.concat([self.df_rooms, new_room], ignore_index=True)
        self.update_tfidf() 

    def recommend_rooms(self, user_skills, user_position, user_tracks, top_n=5):
        user_profile = f"{user_skills} {user_position} {user_tracks}"
        user_vector = self.vectorizer.transform([user_profile])
        similarities = cosine_similarity(user_vector, self.tfidf_matrix).flatten()
        recommended_indices = similarities.argsort()[-top_n:][::-1]
        return self.df_rooms.iloc[recommended_indices][["ID", "Name", "Skills", "Position", "Tracks"]]

file_path = r"C:\Users\sara\Downloads\RoomData.xlsx"
rec_system = RecommendationSystem(file_path)

rec_system.add_new_room(100, "Network Room", "R, Network", "Researcher", "Network,cybersecurity")
rec_system.add_new_room(200, "Room2", " Network", "Ai", "Network,cybersecurity")

user_skills = "R"
user_position = "Network engineer"
user_tracks = "Networking"
recommended_rooms = rec_system.recommend_rooms(user_skills, user_position, user_tracks)

def style_dataframe(df):
    return df.style.set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-size', '14px')]},
        {'selector': 'td', 'props': [('font-size', '13px')]},
    ]).set_properties(**{'text-align': 'center'})

style_dataframe(recommended_rooms)


,ID,Name,Skills,Position,Tracks
1001,200,Room2,Network,Ai,"Network,cybersecurity"
1000,100,Network Room,"R, Network",Researcher,"Network,cybersecurity"
289,5367b470-0fbf-46c9-b3c5-0bcff441608d,Project-4491,C#,DevOps Engineer,Cybersecurity
940,f8c2884e-cfc9-4cdc-8274-d0526f8f4404,Project-4598,C#,DevOps Engineer,Cybersecurity
620,120434e8-f5e0-47c6-93cc-9d266ca3e86d,Project-2808,C#,DevOps Engineer,Mobile Development
